In [2]:
import torch
import psutil
import matplotlib.pyplot as plt
import pickle
from Questionnaire import Questionnaire
from Vocabulary import Vocabulary
from QuestionsParser import QuestionsParser

import settings
from trainer_helpers import train_cbow, find_hyperparams

In [3]:
if settings.USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cpu


In [4]:
with open(settings.test_cbow_data_generator_path, 'rb') as f:
    data_generator = pickle.load(f)

In [5]:
with open(settings.test_questions_parser_path, 'rb') as f:
    questions_parser = pickle.load(f)

In [6]:
unformated_training_data = [
(('mother', 'does', 'female', 'woman'), 'she'), 
(('does', 'female', 'mother', 'woman'), 'she'), 
(('her', 'female', 'he', 'woman'), 'she'), 
(('does', 'she', 'mother', 'woman'), 'female'), 
(('she', 'her', 'woman', 'mother'), 'female'), 
(('she', 'male', 'woman', 'mother'), 'female'), 
(('she', 'does', 'female', 'mother'), 'woman'), 
(('female', 'she', 'mother', 'her'), 'woman'), 
(('female', 'she', 'man', 'does'), 'woman'), 

(('father', 'does', 'male', 'man'), 'he'), 
(('does', 'male', 'father', 'man'), 'he'), 
(('his', 'male', 'she', 'man'), 'he'), 
(('does', 'he', 'father', 'man'), 'male'), 
(('he', 'his', 'man', 'father'), 'male'), 
(('he', 'female', 'man', 'father'), 'male'), 
(('he', 'does', 'male', 'father'), 'man'), 
(('male', 'he', 'father', 'his'), 'man'), 
(('male', 'he', 'woman', 'does'), 'man'), 
]

In [7]:
corpus = [['mother', 'she', 'woman', 'female', 
'father', 'he', 'man', 'male', 'his', 'does', 'her']]
vocabulary = Vocabulary(corpus)

In [8]:
training_data = []
for X, y in unformated_training_data:
    X_idx = [vocabulary.get_idx(x) for x in X]
    y_idx = vocabulary.get_idx(y)
    sample = (X_idx, y_idx)
    training_data.append(sample)

In [9]:
print(training_data[:3])

[([0, 9, 3, 2], 1), ([9, 3, 0, 2], 1), ([10, 3, 5, 2], 1)]


In [10]:
questions_parser = QuestionsParser(vocabulary, settings.test_questions_filename, settings.test_meta_lines_indices)

In [11]:
lr = 10**(-1.5)
gamma = 10**(-0.05)
step_size = 3
model_path = 'models_weights/cbow'
train_losses, val_accuracies, model = train_cbow(training_data, questions_parser, settings.embedding_dim, device, lr,
gamma, step_size, model_path)

questionnaire has setup
setup completed
{'total': 8589934592, 'available': 2871586816, 'percent': 66.6, 'used': 4732493824, 'free': 58937344, 'active': 2814943232, 'inactive': 2591404032, 'wired': 1917550592}
0.0
epoch: 0
loss: 0.599075436592102
epoch: 1
loss: 0.2627752721309662
epoch: 2
loss: 0.12606829032301903
epoch: 3
loss: 0.06648122891783714
epoch: 4
loss: 0.03391628433018923
0.0
epoch: 5
loss: 0.01244700257666409
epoch: 6
loss: 0.009988484205678105
epoch: 7
loss: 0.0023761625634506345
epoch: 8
loss: 0.0027840229449793696
epoch: 9
loss: 0.0014698424492962658
0.0
epoch: 10
loss: 0.0007144494011299685
epoch: 11
loss: 0.0007970811275299639
epoch: 12
loss: 0.0005388766003306955
epoch: 13
loss: 0.0003782146086450666
epoch: 14
loss: 0.0004772784886881709
0.0
epoch: 15
loss: 0.0002490434271749109
epoch: 16
loss: 0.00028081723212380894
epoch: 17
loss: 0.00035814059083350003
epoch: 18
loss: 0.00023894939295132644
epoch: 19
loss: 0.00018427377290208824
0.0
epoch: 20
loss: 0.000269875188678

In [12]:
a = questionnaire.check_val_accuracy()
print(a)

NameError: name 'questionnaire' is not defined

In [15]:
X = ('her', 'female', 'he', 'woman')
idx = [vocabulary.get_idx(x) for x in X]
idx = torch.tensor(idx)
print(idx)
model.batch_size = 1
log_prob = model(idx)
print(log_prob)
answer_idx = log_prob.argmax().item()
print(answer_idx)
answer = vocabulary.get_word(answer_idx)
print(answer)

tensor([10,  3,  5,  2])
tensor([[-2.7533e+01, -2.8729e-05, -1.4590e+01, -1.0794e+01, -2.7739e+01,
         -2.0181e+01, -1.9511e+01, -1.1764e+01, -2.6923e+01, -2.8557e+01,
         -2.8527e+01]], grad_fn=<LogSoftmaxBackward>)
1
she
